In [9]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv("music-listening-data.csv")

In [13]:
df.head()

,id,timestamp,attention-class,class-score,model-version,track-title,track-id,artist,text-reading
0,1,2024-12-22 16:29:33.938818+00,DISTRACTED,0.866439,attention-detection-v4,Love is Heartbreak,7FyDi2JX5dyXfcN0Kepey7,Cousin Tony's Brand New Firebird,483
1,2,2024-12-22 16:29:41.293985+00,DISTRACTED,0.876667,attention-detection-v4,Love is Heartbreak,7FyDi2JX5dyXfcN0Kepey7,Cousin Tony's Brand New Firebird,407
2,3,2024-12-22 16:29:45.302946+00,FOCUSED,0.975039,attention-detection-v4,Love is Heartbreak,7FyDi2JX5dyXfcN0Kepey7,Cousin Tony's Brand New Firebird,107
3,4,2024-12-22 16:29:48.667685+00,FOCUSED,0.961814,attention-detection-v4,Love is Heartbreak,7FyDi2JX5dyXfcN0Kepey7,Cousin Tony's Brand New Firebird,196
4,5,2024-12-22 16:29:51.933117+00,FOCUSED,0.962477,attention-detection-v4,Love is Heartbreak,7FyDi2JX5dyXfcN0Kepey7,Cousin Tony's Brand New Firebird,195


In [77]:
df = df.drop(labels=["id","timestamp","model-version","track-id"], axis=1)

In [ ]:
tracks = df["track-title"]
artists= df["artist"]

# unique_tracks = df["track-title"].unique()
# unique_artists = df["artist"].unique()
# was going to use this to get all the songs I need to find metadata for but if there are tracks from the same artist it might cause problems

In [34]:
# https://www.geeksforgeeks.org/python-iterate-multiple-lists-simultaneously/
track_dict = {}
for (track, artist) in zip(tracks,artists):
    track_dict[track] = artist
# the end result will only contain unique keys for each song - artist repeats will not be impacted

In [28]:
from bs4 import BeautifulSoup
import re

import requests

import os


def prepare_url(artist, song):
    artist_lower = artist.lower()
    artist_clean = re.sub(
        r"[^a-zA-Z0-9 ]", "", artist_lower
    )  # remove non alphanumeric characters + keep spaces using a regular expression
    # reference: https://flexiple.com/python/remove-non-alphanumeric-characters-python

    artist_url = artist_clean.replace(" ", "-").capitalize()  # convert to kebab case

    song_lower = song.lower()
    song_clean = re.sub(
        r"[^a-zA-Z0-9 ]", "", song_lower
    )  # remove non alphanumeric characters + keep spaces
    song_url = song_clean.replace(" ", "-")  # convert to kebab case

    # prepre for genius format:
    url = "https://genius.com/" + artist_url + "-" + song_url + "-lyrics"

    return url


def get_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, "html.parser")
    lyrics_divs = html.find_all("div", attrs={"data-lyrics-container": "true"})

    if not lyrics_divs:
        print(f"Could not find lyrics for {url}")
        return None

    lyrics = "\n".join([div.get_text(separator="\n") for div in lyrics_divs])
    lyrics = re.sub(r"[\(\[].*?[\)\]]", "", lyrics)
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])

    return lyrics
    # reference: https://medium.com/@rachit.lsoni/scraping-song-lyrics-a-fun-and-practical-guide-c0b07e8e7312


def compute_lyricalness(artist, song):
    url = prepare_url(artist, song)
    lyrics = get_lyrics(url)

    if lyrics:
        # for initial implementation use simple measure of number of words per song
        return len(lyrics.split())
    else:
        return None

In [35]:
track_dict

{'Love is Heartbreak': "Cousin Tony's Brand New Firebird",
 'Thought We Were Falling In Love': 'Vona Vella',
 'Garden of the Sun': 'ØZWALD',
 'better.': 'Grand Lotus',
 "I'LL DO ANYTHING": 'Comfort Royale',
 'Motels': 'Royel Otis',
 'Hard Times for Heatherhead': 'Generationals',
 'Mean Streets': 'Tennis',
 'How Many Times Have You Driven By': 'Hana Vu',
 'What Was Mine': 'Sarah Kinsley',
 'Slugger': 'SASAMI',
 'Mystic': 'Evangeline',
 'Everyone Adores You (at least I do)': 'Matt Maltese',
 'Witching Hour': 'TOPS',
 'The Hustle': 'Kiltro',
 "Baby Don't Believe": 'Tennis',
 'Timothy': 'Tennis',
 'The Bend': 'King Princess',
 'Smile For The Camera': 'Etta Marcus',
 'Heels To The Moon': 'Gengahr',
 'I Know The End': 'Phoebe Bridgers',
 'CooCool': 'Róisín Murphy, DJ Koze',
 'Life in a Bubble I Blew - bonus track': 'Joywave',
 'This Is What They Meant': 'Biig Piig',
 'Independent': 'Salt-N-Pepa',
 'Elephant in the Room': 'Kynsy',
 'Bad Habit': 'Steve Lacy',
 'Always': 'The Snuts',
 "Baby Don

In [37]:
lyricalness_df = {}

for track, artist in track_dict.items():
    lyricalness_df[track] = compute_lyricalness(artist, track)

Could not find lyrics for https://genius.com/Grand-lotus-better-lyrics
Could not find lyrics for https://genius.com/Comfort-royale-ill-do-anything-lyrics
Could not find lyrics for https://genius.com/Royel-otis-motels-lyrics
Could not find lyrics for https://genius.com/Evangeline-mystic-lyrics
Could not find lyrics for https://genius.com/Risn-murphy-dj-koze-coocool-lyrics
Could not find lyrics for https://genius.com/Joywave-life-in-a-bubble-i-blew--bonus-track-lyrics
Could not find lyrics for https://genius.com/Saltnpepa-independent-lyrics
Could not find lyrics for https://genius.com/The-clash-train-in-vain-stand-by-me--remastered-lyrics
Could not find lyrics for https://genius.com/Loyle-carner-tom-misch-angel-lyrics
Could not find lyrics for https://genius.com/Ms-lauryn-hill-lost-ones-lyrics
Could not find lyrics for https://genius.com/Gorillaz-rehash-lyrics
Could not find lyrics for https://genius.com/William-devaughn-be-thankful-for-what-youve-got--david-todd-remix-radio-edit-lyrics


In [39]:
lyricalness_df

{'Love is Heartbreak': 178,
 'Thought We Were Falling In Love': 187,
 'Garden of the Sun': 163,
 'better.': None,
 "I'LL DO ANYTHING": None,
 'Motels': None,
 'Hard Times for Heatherhead': 154,
 'Mean Streets': 223,
 'How Many Times Have You Driven By': 89,
 'What Was Mine': 241,
 'Slugger': 442,
 'Mystic': None,
 'Everyone Adores You (at least I do)': 111,
 'Witching Hour': 197,
 'The Hustle': 270,
 "Baby Don't Believe": 321,
 'Timothy': 269,
 'The Bend': 312,
 'Smile For The Camera': 172,
 'Heels To The Moon': 125,
 'I Know The End': 326,
 'CooCool': None,
 'Life in a Bubble I Blew - bonus track': None,
 'This Is What They Meant': 256,
 'Independent': None,
 'Elephant in the Room': 280,
 'Bad Habit': 376,
 'Always': 276,
 "Baby Don't Cry": 249,
 'Future Starts Slow': 249,
 'Train in Vain (Stand by Me) - Remastered': None,
 'Old School': 284,
 'Wait a Minute!': 304,
 'Home Again': 109,
 'Places And Spaces': 171,
 'Little Dark Age': 355,
 'Angel': None,
 'Lost Ones': None,
 'Re-Hash': 

In [40]:
df_track = []
df_lyric_count = []
for track, lyric_count in lyricalness_df.items():
    df_track.append(artist)
    df_lyric_count.append(lyric_count)

In [42]:
df_dict = {
    "track": df_track,
    "lyric-count": df_lyric_count
}

lyric_df = pd.DataFrame(df_dict)
lyric_df.to_csv("lyric-data.csv")

In [44]:
lyric_df.head()

,artist,lyric-count
0,Love is Heartbreak,178.0
1,Thought We Were Falling In Love,187.0
2,Garden of the Sun,163.0
3,better.,NaN
4,I'LL DO ANYTHING,NaN


In [49]:
cleaned_lyrics = lyric_df.dropna()

In [50]:
lyric_count_list = cleaned_lyrics["lyric-count"]

In [51]:
avg = np.average(cleaned_lyrics["lyric-count"])

In [52]:
avg

np.float64(274.42857142857144)

Average Lyric Count is 274.42857142857144 (round to 275)
We can use this to fill in any data gaps in the original lyrics data - this will allow us to have more data for training - however the method is not very sophisticated and will cause inaccuracies in our predictions. Another approach could be to get an average of the specific artists lyrics and use that to approximate the lyrics for that given song.

In [73]:
# https://saturncloud.io/blog/how-to-replace-none-with-nan-in-pandas-dataframe/
filled_lyric_df = lyric_df.fillna(value=275)
filled_lyric_df.to_csv("lyric-data-filled.csv")

In [55]:
import deezer
import requests

client = deezer.Client()

def get_song_bpm(track, artist):
    try:
        result = client.search(track=track, artist=artist)

        if result:
            print(f"getting bpm for {track} by {artist}")
            bpm = result[0].bpm
            if bpm:
                return bpm
            else:
                print("BPM not found")
                return None
        else:
            print("Song not found")
            return None

    # GPT-4 was used to generate error handling code here:
    except requests.exceptions.HTTPError as http_err:
        if http_err.response.status_code == 403:
            print(
                "403 Error: Access Forbidden. Please check your API key or authentication."
            )
        else:
            print(f"HTTP error occurred: {http_err}")  # For other HTTP errors
    except Exception as err:
        print(f"An error occurred: {err}")

In [56]:
track_dict

{'Love is Heartbreak': "Cousin Tony's Brand New Firebird",
 'Thought We Were Falling In Love': 'Vona Vella',
 'Garden of the Sun': 'ØZWALD',
 'better.': 'Grand Lotus',
 "I'LL DO ANYTHING": 'Comfort Royale',
 'Motels': 'Royel Otis',
 'Hard Times for Heatherhead': 'Generationals',
 'Mean Streets': 'Tennis',
 'How Many Times Have You Driven By': 'Hana Vu',
 'What Was Mine': 'Sarah Kinsley',
 'Slugger': 'SASAMI',
 'Mystic': 'Evangeline',
 'Everyone Adores You (at least I do)': 'Matt Maltese',
 'Witching Hour': 'TOPS',
 'The Hustle': 'Kiltro',
 "Baby Don't Believe": 'Tennis',
 'Timothy': 'Tennis',
 'The Bend': 'King Princess',
 'Smile For The Camera': 'Etta Marcus',
 'Heels To The Moon': 'Gengahr',
 'I Know The End': 'Phoebe Bridgers',
 'CooCool': 'Róisín Murphy, DJ Koze',
 'Life in a Bubble I Blew - bonus track': 'Joywave',
 'This Is What They Meant': 'Biig Piig',
 'Independent': 'Salt-N-Pepa',
 'Elephant in the Room': 'Kynsy',
 'Bad Habit': 'Steve Lacy',
 'Always': 'The Snuts',
 "Baby Don

In [59]:
bpm_dict = {}
for track, artist in track_dict.items():
    bpm_dict[track] = get_song_bpm(track, artist)

getting bpm for Love is Heartbreak by Cousin Tony's Brand New Firebird
BPM not found
getting bpm for Thought We Were Falling In Love by Vona Vella
BPM not found
getting bpm for Garden of the Sun by ØZWALD
BPM not found
getting bpm for better. by Grand Lotus
BPM not found
getting bpm for I'LL DO ANYTHING by Comfort Royale
BPM not found
getting bpm for Motels by Royel Otis
BPM not found
getting bpm for Hard Times for Heatherhead by Generationals
BPM not found
getting bpm for Mean Streets by Tennis
BPM not found
Song not found
getting bpm for What Was Mine by Sarah Kinsley
BPM not found
getting bpm for Slugger by SASAMI
BPM not found
getting bpm for Mystic by Evangeline
BPM not found
getting bpm for Everyone Adores You (at least I do) by Matt Maltese
BPM not found
getting bpm for Witching Hour by TOPS
BPM not found
getting bpm for The Hustle by Kiltro
BPM not found
getting bpm for Baby Don't Believe by Tennis
getting bpm for Timothy by Tennis
getting bpm for The Bend by King Princess
BPM 

In [62]:
tracks = []
bpms = []
for track, bpm in bpm_dict.items():
    tracks.append(track)
    bpms.append(bpm)

In [67]:
bpm_prep = {
    "track": tracks,
    "bpm": bpms
}

bpm_df = pd.DataFrame(bpm_prep)
bpm_df.to_csv("bpm-data.csv")

In [70]:
# https://www.geeksforgeeks.org/python-remove-none-values-from-list/

clean_bpms = [x for x in bpms if x is not None]
avg = np.average(clean_bpms)
print(avg)

133.28170731707314


Average BPM is 133.28170731707314 (round to 135)

In [74]:
filled_bpm_df = bpm_df.fillna(value=135)

In [75]:
filled_bpm_df.to_csv("bpm-data-filled.csv")

# Prepare the main dataset

In [78]:
df.head()

,attention-class,class-score,track-title,artist,text-reading
0,DISTRACTED,0.866439,Love is Heartbreak,Cousin Tony's Brand New Firebird,483
1,DISTRACTED,0.876667,Love is Heartbreak,Cousin Tony's Brand New Firebird,407
2,FOCUSED,0.975039,Love is Heartbreak,Cousin Tony's Brand New Firebird,107
3,FOCUSED,0.961814,Love is Heartbreak,Cousin Tony's Brand New Firebird,196
4,FOCUSED,0.962477,Love is Heartbreak,Cousin Tony's Brand New Firebird,195


In [81]:
lyric_dataset=pd.read_csv("lyric-data-filled.csv")
tracks = lyric_dataset["track"]
lyrics = lyric_dataset["lyric-count"]

lyrics_reference = {}

for track, lyric_count in zip(tracks, lyrics):
    lyrics_reference[track] = lyric_count

In [83]:
lyrics_reference

{'Love is Heartbreak': 178.0,
 'Thought We Were Falling In Love': 187.0,
 'Garden of the Sun': 163.0,
 'better.': 275.0,
 "I'LL DO ANYTHING": 275.0,
 'Motels': 275.0,
 'Hard Times for Heatherhead': 154.0,
 'Mean Streets': 223.0,
 'How Many Times Have You Driven By': 89.0,
 'What Was Mine': 241.0,
 'Slugger': 442.0,
 'Mystic': 275.0,
 'Everyone Adores You (at least I do)': 111.0,
 'Witching Hour': 197.0,
 'The Hustle': 270.0,
 "Baby Don't Believe": 321.0,
 'Timothy': 269.0,
 'The Bend': 312.0,
 'Smile For The Camera': 172.0,
 'Heels To The Moon': 125.0,
 'I Know The End': 326.0,
 'CooCool': 275.0,
 'Life in a Bubble I Blew - bonus track': 275.0,
 'This Is What They Meant': 256.0,
 'Independent': 275.0,
 'Elephant in the Room': 280.0,
 'Bad Habit': 376.0,
 'Always': 276.0,
 "Baby Don't Cry": 249.0,
 'Future Starts Slow': 249.0,
 'Train in Vain (Stand by Me) - Remastered': 275.0,
 'Old School': 284.0,
 'Wait a Minute!': 304.0,
 'Home Again': 109.0,
 'Places And Spaces': 171.0,
 'Little Da

In [84]:
lyric_column = []

total_tracks = df["track-title"]

for track in total_tracks:
    lyric_column.append(lyrics_reference[track])

In [87]:
bpm_dataset = pd.read_csv("bpm-data-filled.csv")
tracks = bpm_dataset["track"]
bpms = bpm_dataset["bpm"]

bpm_reference = {}

for track, bpm_measure in zip(tracks, bpms):
    bpm_reference[track] = bpm_measure

In [88]:
bpm_reference

{'Love is Heartbreak': 135.0,
 'Thought We Were Falling In Love': 135.0,
 'Garden of the Sun': 135.0,
 'better.': 135.0,
 "I'LL DO ANYTHING": 135.0,
 'Motels': 135.0,
 'Hard Times for Heatherhead': 135.0,
 'Mean Streets': 135.0,
 'How Many Times Have You Driven By': 135.0,
 'What Was Mine': 135.0,
 'Slugger': 135.0,
 'Mystic': 135.0,
 'Everyone Adores You (at least I do)': 135.0,
 'Witching Hour': 135.0,
 'The Hustle': 135.0,
 "Baby Don't Believe": 159.01,
 'Timothy': 149.3,
 'The Bend': 135.0,
 'Smile For The Camera': 135.0,
 'Heels To The Moon': 135.0,
 'I Know The End': 135.0,
 'CooCool': 135.0,
 'Life in a Bubble I Blew - bonus track': 130.01,
 'This Is What They Meant': 135.0,
 'Independent': 100.11,
 'Elephant in the Room': 135.0,
 'Bad Habit': 135.0,
 'Always': 135.0,
 "Baby Don't Cry": 135.0,
 'Future Starts Slow': 128.0,
 'Train in Vain (Stand by Me) - Remastered': 122.3,
 'Old School': 156.01,
 'Wait a Minute!': 101.08,
 'Home Again': 137.8,
 'Places And Spaces': 184.6,
 'Lit

In [89]:
bpm_column = []

total_tracks = df["track-title"]

for track in total_tracks:
    bpm_column.append(bpm_reference[track])

In [ ]:
bpm_column

In [91]:
df["bpm"] = bpm_column
df["lyric-count"] = lyric_column

In [92]:
df.to_csv("music-listening-data-with-metadata.csv")